# Deep learning algorithms to classify audio (EfficientNet B2 + Two-stage fine-tuning)

In [3]:
import gc
import glob
import logging
import os
import random
import re
import sys
import time
import warnings
from collections import Counter
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    f1_score,
    roc_auc_score,
)
from sklearn.model_selection import (
    RandomizedSearchCV,
    StratifiedKFold,
    train_test_split,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB2, ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)

os.environ["CUDA_VISIBLE_DEVICES"] = ""

print(tf.__version__)
print(dir(tf.keras))

2.19.0
['DTypePolicy', 'FloatDTypePolicy', 'Function', 'Initializer', 'Input', 'InputSpec', 'KerasTensor', 'Layer', 'Loss', 'Metric', 'Model', 'Operation', 'Optimizer', 'Quantizer', 'Regularizer', 'RematScope', 'Sequential', 'StatelessScope', 'SymbolicScope', 'Variable', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'activations', 'applications', 'backend', 'callbacks', 'config', 'constraints', 'datasets', 'device', 'distribution', 'dtype_policies', 'export', 'initializers', 'layers', 'legacy', 'losses', 'metrics', 'mixed_precision', 'models', 'name_scope', 'ops', 'optimizers', 'preprocessing', 'quantizers', 'random', 'regularizers', 'remat', 'tree', 'utils', 'version', 'visualization', 'wrappers']


## BirdCLEF 2025: EfficientNet B2 + Two-stage fine-tuning Multi-label Classification Approach

### Overview

To improve model performance beyond the ResNet50 baseline, we introduced **EfficientNet B2** as the feature extractor. EfficientNet models are known for their compound scaling of depth, width, and resolution, offering higher performance with fewer parameters.

Instead of training the entire network from scratch, we adopt a **two-stage fine-tuning strategy** to leverage pretrained weights from ImageNet and prevent overfitting on our limited dataset.

---

### Stage 1: Freeze the base model

- Load `EfficientNetB2` with `weights='imagenet'` and `include_top=False`
- **Freeze all layers** in the base model (i.e., `base_model.trainable = False`)
- Only train the **newly added top layers**:
  - GlobalAveragePooling → Dropout → Dense(206, activation='sigmoid')
- Use `BinaryCrossentropy` with `label_smoothing` to stabilize training

### Stage 2: Unfreeze and fine-tune

- After the top layers are warm-started, **unfreeze the last N layers** of EfficientNet
- Recompile the model with a **lower learning rate** (e.g., 1e-5)
- Continue training for another 5–10 epochs

---

### Why EfficientNet B2?

| Model       | Params | Accuracy | Speed | Notes                |
|-------------|--------|----------|-------|----------------------|
| ResNet50    | 24M    | High     | OK    | Strong baseline      |
| **EffNetB2**| 8M     | High+    | Fast  | Higher AUC with less |
| EffNetB3    | 12M    | Higher   | Slower| GPU memory ↑         |

EfficientNet-B2 offers **better performance-per-parameter ratio** than ResNet50, making it a solid upgrade path.

In [2]:
# -------------------------
# 1) Load train_data.npy
# -------------------------
# Content example:
# data_dict[fid] = {
# 'data': (128,256) Mel spectrum,
# 'label': 'Name of a species'
# }
# -------------------------
data_dict = np.load('dataset/train_data.npy', allow_pickle=True).item()

X_list = []
y_list = []

all_labels_set = set()

for fid, content in data_dict.items():
    mel_2d = content['data']             # shape=(128,256)
    label_str = content['label']         # 'species_xxx'

    X_list.append(mel_2d)
    y_list.append(label_str)
    all_labels_set.add(label_str)

X_array = np.array(X_list, dtype=np.float32)    # shape=(N,128,256)
y_array = np.array(y_list)                      # shape=(N,)

all_labels = sorted(list(all_labels_set))
label_to_idx = {lb: i for i, lb in enumerate(all_labels)}
num_species = len(all_labels)

print("Number of samples:", X_array.shape[0])
print("Mel shape: (128,256)")
print("Number of unique species:", num_species)

Number of samples: 28564
Mel shape: (128,256)
Number of unique species: 206


In [5]:
# -----------------------------
# 2) Multi-label One-Hot: Only one position in each record is 1
# -----------------------------
Y_one_hot = np.zeros((len(y_array), num_species), dtype=np.float32)
for i, lb in enumerate(y_array):
    Y_one_hot[i, label_to_idx[lb]] = 1.0

# -----------------------------
# 3) Split training/validation set (80/20)
# -----------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X_array, Y_one_hot, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_array # Stratify by string label
)

print("Train shape:", X_train.shape, y_train.shape)
print("Val shape:",   X_val.shape,   y_val.shape)

Train shape: (22851, 128, 256) (22851, 206)
Val shape: (5713, 128, 256) (5713, 206)


In [6]:
# -----------------------------
# 4) Dealing with data imbalance -> class_weight
# Since each record has only one label, we can count the number of times each label appears
# and assign values ​​in reverse proportion.
# -----------------------------
label_counts = Counter(y_array)
max_count = max(label_counts.values())
# Give higher weight to less common categories
class_weight = {}
for lb, freq in label_counts.items():
    idx = label_to_idx[lb]
    class_weight[idx] = max_count / freq

print("Class weight example:", list(class_weight.items())[:5])

Class weight example: [(110, 7.7952755905511815), (177, 6.470588235294118), (71, 3.1832797427652735), (30, 33.0), (47, 47.142857142857146)]


In [7]:
# -----------------------------
# 5) Build data pipeline + data augmentation
# Random flip/rotate (for images)
# -----------------------------
augment_layers = tf.keras.Sequential([
    layers.RandomFlip(mode='horizontal'),
    layers.RandomRotation(0.1),
], name="data_augmentation")

def preprocess_fn(x, y):
    # x: (128,256) => expand dims到(128,256,1)
    x = tf.expand_dims(x, axis=-1)
    # cast to float
    x = tf.cast(x, tf.float32)
    x = augment_layers(x, training=True)  
    return x, y

def preprocess_fn_val(x, y):
    # No data augmentation for validation set
    x = tf.expand_dims(x, axis=-1)
    x = tf.cast(x, tf.float32)
    return x, y

batch_size = 16
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(buffer_size=2048).map(preprocess_fn).batch(batch_size).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_ds = val_ds.map(preprocess_fn_val).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [10]:
# -----------------------------
# 6) Build effnet_b2 (Keras)
# -----------------------------
def build_effnet_b2(input_shape=(128, 256, 1), num_classes=206, drop_rate=0.3):
    inp = layers.Input(shape=input_shape)
    x = layers.Conv2D(3, (1, 1), padding='same')(inp)

    base = EfficientNetB2(
        include_top=False,
        weights='imagenet'
    )
    base.trainable = False

    x = base(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(drop_rate)(x)
    out = layers.Dense(num_classes, activation='sigmoid')(x)

    return models.Model(inp, out, name="EffNetB2_BirdCLEF")


model = build_effnet_b2()

loss_fn = tf.keras.losses.BinaryCrossentropy(
    from_logits=False,
    label_smoothing=0.05  # Smoothing
)

metrics = [
    tf.keras.metrics.AUC(name='auc', multi_label=True),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer=Adam(1e-4), loss=loss_fn, metrics=metrics)

model.summary()

# Early Stop Callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

Model: "EffNetB2_BirdCLEF"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 128, 256, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 256, 3)    │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb2 (Functional)     │ (None, 4, 8, 1408)     │     7,768,569 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1408)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 206)            │       290,254 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,058,829 (30.74 MB)

 Trainable params: 290,260 (1.11 MB)

 Non-trainable params: 7,768,569 (29.63 MB)

In [ ]:
# stage 1
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[early_stopping, lr_scheduler]
)

Epoch 1/20
  75/1429 ━━━━━━━━━━━━━━━━━━━━ 5:40 251ms/step - auc: 0.3081 - loss: 0.5761 - precision: 0.0057 - recall: 0.2421

In [ ]:
# stage 2
for layer in model.get_layer('efficientnetb2').layers[-100:]:
    layer.trainable = True

model.compile(optimizer=Adam(1e-5), loss=loss_fn, metrics=metrics)
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[early_stopping, lr_scheduler]
)

model.save("effnetb2_two_stage.h5")

In [ ]:
results = model.evaluate(val_ds, verbose=1)
val_loss = results[0]
val_auc = results[1]
val_precision = results[2]
val_recall = results[3]

print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation AUC: {val_auc:.4f}")
print(f"Precision: {val_precision:.4f}")
print(f"Recall: {val_recall:.4f}")

358/358 ━━━━━━━━━━━━━━━━━━━━ 54s 150ms/step - auc: 0.7709 - loss: 0.1266 - precision: 0.7917 - recall: 0.3850
Validation Loss: 0.1266
Validation AUC: 0.8828
Precision: 0.7959
Recall: 0.3898
